In [1]:
import evaluate
from datasets import DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForMultipleChoice,
    TrainingArguments,
    Trainer,
)

d:\IDE\Anaconda\envs\fproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
c3 = DatasetDict.load_from_disk('./c3/')
c3

DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 1625
    })
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

In [3]:
c3['train'][0]

{'id': 0,
 'context': ['男：你今天晚上有时间吗?我们一起去看电影吧?', '女：你喜欢恐怖片和爱情片，但是我喜欢喜剧片，科幻片一般。所以……'],
 'question': '女的最喜欢哪种电影?',
 'choice': ['恐怖片', '爱情片', '喜剧片', '科幻片'],
 'answer': '喜剧片'}

In [4]:
c3.pop("test")

Dataset({
    features: ['id', 'context', 'question', 'choice', 'answer'],
    num_rows: 1625
})

In [5]:
c3

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained('../chinese-macbert-base/')
tokenizer

BertTokenizerFast(name_or_path='../chinese-macbert-base/', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [7]:
c3['train']['context'][1]

['男：足球比赛是明天上午八点开始吧?', '女：因为天气不好，比赛改到后天下午三点了。']

In [8]:
def process_function(examples):
    context = []
    question_choice = []
    labels = []
    for idx in range(len(examples["context"])):
        ctx = "\n".join(examples["context"][idx])
        question = examples["question"][idx]
        choices = examples["choice"][idx]
        for choice in choices:
            context.append(ctx)
            question_choice.append(question + " " + choice)
        if len(choices) < 4:
            for _ in range(4 - len(choices)):
                context.append(ctx)
                question_choice.append(question + " " + "不知道")
        labels.append(choices.index(examples["answer"][idx]))
    tokenized_examples = tokenizer(
        context,
        question_choice,
        truncation="only_first",
        max_length=256,
        padding="max_length",
    )
    tokenized_examples = {
        k: [v[i : i + 4] for i in range(0, len(v), 4)]
        for k, v in tokenized_examples.items()
    }
    tokenized_examples["labels"] = labels
    return tokenized_examples

In [9]:
tokenized_c3 = c3.map(process_function, batched=True)
tokenized_c3

Map:   0%|          | 0/3816 [00:00<?, ? examples/s]

Map: 100%|██████████| 3816/3816 [00:02<00:00, 1861.69 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3816
    })
})

In [10]:
model = AutoModelForMultipleChoice.from_pretrained("../chinese-macbert-base/")

Some weights of the model checkpoint at ../chinese-macbert-base/ were not used when initializing BertForMultipleChoice: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkp

In [11]:
import numpy as np

accuracy = evaluate.load("accuracy")


def compute_metric(pred):
    predcitions, labels = pred
    predcitions = np.argmax(predcitions, axis=-1)
    return accuracy.compute(predictions=predcitions, references=labels)

In [12]:
args = TrainingArguments(
    output_dir="./mutiple_choice",
    per_device_train_batch_size=1,  # 训练时的batch_size
    gradient_accumulation_steps=16,  # *** 梯度累加 ***
    gradient_checkpointing=True,  # *** 梯度检查点 ***
    optim="adafactor",  # *** adafactor优化器 ***
    per_device_eval_batch_size=1,  # 验证时的batch_size
    num_train_epochs=1,  # 训练轮数
    logging_steps=10,  # log 打印的频率
    evaluation_strategy="epoch",  # 评估策略
    save_strategy="epoch",  # 保存策略
    save_total_limit=3,  # 最大保存数
    load_best_model_at_end=True,
    fp16=True,
)

In [13]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_c3["train"],
    eval_dataset=tokenized_c3["validation"],
    compute_metrics=compute_metric,
)

In [14]:
import torch

if hasattr(torch.cuda, "empty_cache"):
    torch.cuda.empty_cache()
trainer.train()

  1%|▏         | 10/741 [00:15<17:04,  1.40s/it]

{'loss': 1.3899, 'learning_rate': 4.939271255060729e-05, 'epoch': 0.01}


  3%|▎         | 20/741 [00:29<17:01,  1.42s/it]

{'loss': 1.4213, 'learning_rate': 4.878542510121458e-05, 'epoch': 0.03}


  4%|▍         | 30/741 [00:44<16:45,  1.41s/it]

...

100%|██████████| 741/741 [19:57<00:00,  1.45s/it]  

{'eval_loss': 1.3249152898788452, 'eval_accuracy': 0.30660377358490565, 'eval_runtime': 90.3156, 'eval_samples_per_second': 42.252, 'eval_steps_per_second': 42.252, 'epoch': 1.0}


100%|██████████| 741/741 [19:57<00:00,  1.62s/it]

{'train_runtime': 1197.581, 'train_samples_per_second': 9.911, 'train_steps_per_second': 0.619, 'train_loss': 1.375907692027639, 'epoch': 1.0}


TrainOutput(global_step=741, training_loss=1.375907692027639, metrics={'train_runtime': 1197.581, 'train_samples_per_second': 9.911, 'train_steps_per_second': 0.619, 'train_loss': 1.375907692027639, 'epoch': 1.0})

In [16]:
from typing import Any


class MultipleChoicePipeline:
    def __init__(self, model, tokenizer) -> None:
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device

    def preprocess(self, context, question, choices):
        cs, qcs = [], []
        for choice in choices:
            cs.append(context)
            qcs.append(question + " " + choice)
        return tokenizer(
            cs, qcs, truncation="only_first", max_length=256, return_tensors="pt"
        )

    def predict(self, inputs):
        inputs = {k: v.unsqueeze(0).to(self.device) for k, v in inputs.items()}
        return self.model(**inputs).logits

    def postprocess(self, logits, choices):
        predition = torch.argmax(logits, dim=-1).cpu().item()
        return choices[predition]

    def __call__(self, context, question, choices) -> Any:
        inputs = self.preprocess(context, question, choices)
        logits = self.predict(inputs)
        result = self.postprocess(logits, choices)
        return result

In [17]:
pipe = MultipleChoicePipeline(model, tokenizer)

In [18]:
pipe("小明在北京上班", "小明在哪里上班？", ["北京", "上海", "河北", "海南", "河北", "海南"])

'北京'